In [ ]:
pip install downcast

In [ ]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from downcast import reduce
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
calendar_=pd.read_csv('/content/drive//My Drive/CS-1/calendar.csv')
sales_train_evaluation_=pd.read_csv('/content/drive//My Drive/CS-1/sales_train_evaluation.csv')
sell_prices_=pd.read_csv('/content/drive//My Drive/CS-1/sell_prices.csv')

* Replace 'NaN' values by 'no_event' in event columns

In [ ]:
cat=['event_name_1','event_type_1','event_name_2','event_type_2']
for i in cat:
  calendar_[i].fillna('no_event',inplace=True)

* Adding feature 'is_weekend' which tells about that day is weekend or not

In [ ]:
f=lambda x: 1 if x<=2 else 0
#https://stackoverflow.com/questions/21608228/conditional-replace-pandas
calendar_['is_weekend']=calendar_['wday'].map(f) 
calendar_['is_weekend']=calendar_['is_weekend'].astype(np.int8)

* Adding feature 'month_day' which tells day of the month

In [ ]:
m=calendar_["date"].tolist()
m=[i.split("-")[2] for i in m]
calendar_["month_day"]=m
calendar_['month_day']=calendar_['month_day'].astype(np.int8)

* Adding feature 'month_week_number' which tells which week of the month

In [ ]:
#https://stackoverflow.com/questions/3806473/python-week-number-of-the-month
calendar_['month_week_number']=(calendar_['month_day']-1) // 7 + 1 
calendar_['month_week_number']=calendar_['month_week_number'].astype(np.int8)

* Adding feature 'events_per_day' which tells us number of events on particular day

In [ ]:
f=lambda x: 0 if x=='no_event' else 1
#https://stackoverflow.com/questions/21608228/conditional-replace-pandas
calendar_['events_per_day']=calendar_['event_type_1'].map(f) 
#https://www.kite.com/python/answers/how-to-get-the-indices-of-rows-in-a-pandas-dataframe-which-satisfy-a-given-condition-in-python
index=calendar_.index 
indices=index[calendar_['event_type_2']!='no_event'].tolist()
for i in indices:
  calendar_['events_per_day'][i]+=1
calendar_['events_per_day']=calendar_['events_per_day'].astype(np.int8)

In [ ]:
calendar_=reduce(calendar_)

In [ ]:
calendar_.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day
0,2011-01-29,11101,Saturday,1,1,2011,d_1,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,no_event,no_event,no_event,no_event,0,0,0,1,30,5,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,no_event,no_event,no_event,no_event,0,0,0,0,31,5,0


In [ ]:
sales_train_evaluation_=reduce(sales_train_evaluation_)

In [ ]:
sell_prices_=reduce(sell_prices_)

In [ ]:
calendar_['date'].nunique()

1969

* No of days data present is 1969

In [ ]:
sales_train_evaluation_[['id','item_id','dept_id','cat_id','store_id','state_id']].nunique()

id          30490
item_id      3049
dept_id         7
cat_id          3
store_id       10
state_id        3
dtype: int64

* No. of unique:
  
  * id: 30490

  * items: 3049

  * Departments: 7

  * Category: 3

  * Stores: 10

  * States: 3


#Melting

* To make analysis of data in table easier we can reshape the data into a more computer-friendly form using pandas in Python. pandas.melt() is one of the function to do so

* pandas.melt() unpivots a DataFrame from wide format to long format.

* melt() function is useful to massage a DataFrame into a format where one or more columns are identifier variables while all other columns considered measured variables are unpivoted to the row axis leaving just two non-identifier columns variable and value.

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.melt.html
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
sales=pd.melt(sales_train_evaluation_,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='demand')
sales=pd.merge(sales,calendar_,on='d',how='left')
sales=pd.merge(sales,sell_prices_,on=['item_id','store_id','wm_yr_wk'],how='left')

In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,NaN


* Since we got many rows with 'NaN' values in Sell Price 

* Thus replacing 'NaN" in 'sell_price' feature with the mean value 

In [ ]:
sales['sell_price']=sales['sell_price'].fillna(sales.groupby('id')['sell_price'].transform('mean'))

In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,8.281250
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,3.970703
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,2.970703


In [ ]:
sales=reduce(sales)

#Lags

* Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

* Lag is expressed in a time unit & corresponds to the amount of data history we allow the model to use when making the prediction.

* Here we have applied Lags on 'demand' column.

* The maximum Lags taken is 70 days

In [ ]:
#https://stackoverflow.com/questions/20410312/how-to-create-a-lagged-data-structure-using-pandas-dataframe
lags=[28,35,42,49,56,63,70]
for i in tqdm(lags):
  sales['lag_'+str(i)]=sales.groupby(['id'])['demand'].shift(i)

100%|██████████| 7/7 [00:16<00:00,  2.35s/it]


In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,8.281250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,2.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Replacing 'NaN' in 'lags' features with 0

In [ ]:
lags=['lag_28','lag_35','lag_42','lag_49','lag_56','lag_63','lag_70']
for i in lags:
  sales[i]=sales[i].fillna(0) 

In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,8.281250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,3.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,2.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
sales=reduce(sales)

#Rolling-Mean

* Rolling is a very useful operation for time series data.

* Rolling means creating a rolling window with a specified size & perform calculations on data in this window which of course rolls through data

* Here we have computing Rolling-Mean on 'demand' column.

* The maximum Window size taken is 42

In [ ]:
#https://stackoverflow.com/questions/13996302/python-rolling-functions-for-groupby-object
##https://www.geeksforgeeks.org/python-pandas-dataframe-transform/
window=[7,14,28,35,42]
for i in tqdm(window):
  sales['rolling_median_'+str(i)]=sales.groupby(['id'])['demand'].transform(lambda s: s.rolling(i,center=False).median())

100%|██████████| 5/5 [3:16:39<00:00, 2359.91s/it]


* Replacing 'NaN' in 'rolling_ mean' features with 0

In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_median_7,rolling_median_14,rolling_median_28,rolling_median_35,rolling_median_42
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,8.281250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,3.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,2.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
window=['rolling_median_7','rolling_median_14','rolling_median_28','rolling_median_35','rolling_median_42']
for i in window:
  sales[i]=sales[i].fillna(0) 

In [ ]:
sales=reduce(sales)

In [ ]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_median_7,rolling_median_14,rolling_median_28,rolling_median_35,rolling_median_42
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,8.281250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,3.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0,2.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Label-Encoding

* Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form.

* Machine learning algorithms can then decide in a better way on how those labels must be operated.

* It is an important pre-processing step for the structured dataset in supervised learning

In [ ]:
 #https://www.mygreatlearning.com/blog/label-encoding-in-python/
labelencoder=LabelEncoder() 
category=['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id']
for i in tqdm(category):
  sales[i+'_']=labelencoder.fit_transform(sales[i])

100%|██████████| 10/10 [01:24<00:00,  8.45s/it]


* Drop all the categorical columns bcoz we already added coresponding columns with label-encoding 

In [ ]:
sales=sales.drop(['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

In [ ]:
sales.head(3)

,d,demand,date,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_median_7,rolling_median_14,rolling_median_28,rolling_median_35,rolling_median_42,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,1,29,5,0,8.281250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,4,4,2,14370,1437,3,1,0,0
1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,1,29,5,0,3.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,4,4,2,14380,1438,3,1,0,0
2,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,1,29,5,0,2.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,4,4,2,14390,1439,3,1,0,0


In [ ]:
sales=reduce(sales)

* Removed '_' from 'd' column values so that we can convert Categorical feature into Numerical feature easily 

In [ ]:
l=[]
for i in sales['d']:
  l.append(i.split('_')[1])
sales['day']=l
#https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
sales['day']=sales['day'].astype(np.int16) 

#Droping duplicate columns 

* Since 'weekday' is represented by 'wday' & 'd' is represented by 'day'

* We already have 'month','year' thats why 'date' is also duplicate column 

In [ ]:
sales=sales.drop(['d','date','weekday'],axis=1)

In [ ]:
data_final=reduce(sales)

In [ ]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 36 columns):
 #   Column             Dtype  
---  ------             -----  
 0   demand             int16  
 1   wm_yr_wk           int16  
 2   wday               int8   
 3   month              int8   
 4   year               int16  
 5   snap_CA            int8   
 6   snap_TX            int8   
 7   snap_WI            int8   
 8   is_weekend         int8   
 9   month_day          int8   
 10  month_week_number  int8   
 11  events_per_day     int8   
 12  sell_price         float16
 13  lag_28             float16
 14  lag_35             float16
 15  lag_42             float16
 16  lag_49             float16
 17  lag_56             float16
 18  lag_63             float16
 19  lag_70             float16
 20  rolling_median_7   float16
 21  rolling_median_14  float16
 22  rolling_median_28  float16
 23  rolling_median_35  float16
 24  rolling_median_42  float16
 25  event_name_1_   

In [ ]:
with open('/content/drive//My Drive/CS-1/data_final.pkl','wb') as f:
     pickle.dump(data_final,f)